Контрольные вопросы:

1) В чем заключается идея внутреннего внимания?
Идея внутреннего внимания (self-attention) в трансформерах заключается в том, чтобы позволить модели сосредотачиваться на различных частях входной последовательности, когда она обрабатывает каждое слово (или токен). Это важно для понимания контекста и взаимосвязей между словами в предложении. 

2) Что такое Multi-head attention?
Multi-head attention (многоголовое внимание) — это расширение механизма внутреннего внимания, которое позволяет модели одновременно использовать несколько "голов" внимания для обработки информации. Это улучшает способность модели усваивать различные аспекты контекста и позволяет более эффективно извлекать семантические зависимости.

Вместо того чтобы использовать одно представление (вектор) для вычисления внимания, входные векторы Query, Key, и Value разделяются на несколько "голов". Каждая голова имеет свои собственные параметры и обучается независимо, что позволяет изучать разные подпространства.

3) Можно ли использовать отдельно кодировщик и декодировщик трансформера? Для каких целей?
Да, можно испольщовать кодировщик и декадировщик отдельно. 
    Кодировщик используется для задач, где необходимо извлечение информации и понимание входных данных:
    1) Классификация текстов
    2) Аннотация сущностей
    3) Извлечение информации
    4) Понимание контекста

    Декодировщик используется для задач, где необходима  генерация последовательности, например:
    1) Генерация текста
    2) Создание описаний
    3) Диалоговые системы

4) Как представляются исходные данные для модели трансформера?
Представление исходных данных для модели трансформера осуществляется через несколько ключевых шагов:
    1) Токенизация  
    2) Преобразование токенов в числовые идентификаторы
    3) Добавление специальных токенов (необязательно)
    4) Паддинг последовательностей
    5) Кодирование позиций
    6) Векторизация

5) К какому классу задач относится задача генерации текста? Какие основные принципы её решения вам известны?
Задача генерации текста относится к классу задач обработки естественного языка (NLP) и обычно рассматривается как подзадача генерации языка или языкового моделирования. Основная цель генерации текста — создание последовательностей слов, которые имеют смысл и соответствуют определенному контексту или тематике.
   1) Использование языковых моделей (Language Models, LM)
   2) Архитектуры на основе трансформеров
   3) Обучение на больших данных
   4) Стратегии генерации
   5) Контроль качества и согласованности

Практические задания:
1) Решите задачу классификации текста на русском языке (или с применением датасета отличного от предложенного в работе) при помощи модели кодировщика Трансформера.
2) Выполните предобработку набора данных для подачи на вход нейронной сети в задаче машинного перевода.
3) Примените описанные здесь модели RNN и Transformer для задачи перевода (например английский - русский) или генерации текста (ответы на вопросы).
4) Сравните результаты моделей RNN и Transformer.

1) Решите задачу классификации текста на русском языке (или с применением датасета отличного от предложенного в работе) при помощи модели кодировщика Трансформера.

In [1]:

import numpy as np
import pandas as pd 

import os

In [2]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')


In [3]:
df.shape

(20491, 2)

In [4]:
df.isnull().sum()

Review    0
Rating    0
dtype: int64

In [5]:
df.head(10)

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
5,love monaco staff husband stayed hotel crazy w...,5
6,"cozy stay rainy city, husband spent 7 nights m...",5
7,"excellent staff, housekeeping quality hotel ch...",4
8,"hotel stayed hotel monaco cruise, rooms genero...",5
9,excellent stayed hotel monaco past w/e delight...,5


In [6]:
def label(rating):
    if rating < 3:
        return 0
    elif rating == 3:
        return 1
    else:
        return 2

df['Label'] = df['Rating'].apply(label)
df['Label'] = df['Label'].astype(int)

In [7]:
df.head(10)


,Review,Rating,Label
0,nice hotel expensive parking got good deal sta...,4,2
1,ok nothing special charge diamond member hilto...,2,0
2,nice rooms not 4* experience hotel monaco seat...,3,1
3,"unique, great stay, wonderful time hotel monac...",5,2
4,"great stay great stay, went seahawk game aweso...",5,2
5,love monaco staff husband stayed hotel crazy w...,5,2
6,"cozy stay rainy city, husband spent 7 nights m...",5,2
7,"excellent staff, housekeeping quality hotel ch...",4,2
8,"hotel stayed hotel monaco cruise, rooms genero...",5,2
9,excellent stayed hotel monaco past w/e delight...,5,2


In [8]:
from sklearn.model_selection import train_test_split

X = df['Review']
y = df['Label']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, stratify=y, random_state=100)

In [9]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_valid, y_valid], axis=1)

In [10]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [11]:
from transformers import AutoTokenizer

checkpoint = 'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/aa-sablin/env_lab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from datasets import load_dataset

data_files = {
    'train': r'train.csv',
    'test': r'test.csv'
}
dataset = load_dataset('csv', data_files=data_files)


Generating train split: 15368 examples [00:00, 132946.26 examples/s]
Generating test split: 5123 examples [00:00, 137670.15 examples/s]


In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Review', 'Label'],
        num_rows: 15368
    })
    test: Dataset({
        features: ['Review', 'Label'],
        num_rows: 5123
    })
})

In [14]:
def tokenize_function(example):
    return tokenizer(example['Review'], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 5123/5123 [00:00<00:00, 8702.14 examples/s]


In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Review', 'Label', 'input_ids', 'attention_mask'],
        num_rows: 15368
    })
    test: Dataset({
        features: ['Review', 'Label', 'input_ids', 'attention_mask'],
        num_rows: 5123
    })
})

In [16]:
import tensorflow as tf
print(tf.__version__)

2025-05-30 22:27:19.162216: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-30 22:27:19.166775: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-30 22:27:19.173104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748618839.185133   16260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748618839.188566   16260 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748618839.197438   16260 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

2.19.0


In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [ ]:
tf_train_dataset = tokenized_dataset['train'].to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['Label'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=10
)

tf_test_dataset = tokenized_dataset['test'].to_tf_dataset(
    columns=['input_ids', 'attention_mask'],
    label_cols=['Label'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=10
)

/home/aa-sablin/env_lab/lib/python3.12/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
E0000 00:00:1748618840.701374   16260 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1748618840.711840   16260 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries menti

In [19]:
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, GaussianDropout, Bidirectional, Embedding, LSTM
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.models import Model

vocab_size = tokenizer.vocab_size

def build_model(num_classes=3):
    inputs = Input(shape=(None,), dtype='int32', name='input_ids')
    x = Embedding(input_dim=vocab_size, output_dim=50)(inputs)
    x = Bidirectional(LSTM(64))(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model
    

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

def make_callbacks():
    lr_callback = ReduceLROnPlateau(
            monitor='val_loss',     
            factor=0.5,              
            patience=5,              
            verbose=1,               
            min_lr=1e-6            
        )
    
    early_stop = EarlyStopping(
        patience=15, 
        monitor='val_loss', 
        restore_best_weights=True, 
        mode='min'
    )

    checkpoint = ModelCheckpoint(
        filepath="best_model.keras",  
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=0
    )
    
    return [lr_callback, early_stop, checkpoint]


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

dnn = build_model()

optimizer = Nadam(learning_rate=0.0001)
    
dnn.compile(
        optimizer=optimizer,
        loss=SparseCategoricalCrossentropy(),
        metrics=['accuracy']    
    )

dnn.fit(
    tf_train_dataset, 
    validation_data=tf_test_dataset, 
    batch_size=100,
    epochs=5,
    callbacks=make_callbacks(),   
)

test_results = dnn.evaluate(tf_test_dataset)
print("Test Acc.: {:.2f}%".format(test_results[1] * 100))

Epoch 1/5
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 162s 104ms/step - accuracy: 0.7386 - loss: 0.7442 - val_accuracy: 0.8378 - val_loss: 0.4359 - learning_rate: 1.0000e-04
Epoch 2/5
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 163s 106ms/step - accuracy: 0.8561 - loss: 0.3807 - val_accuracy: 0.8528 - val_loss: 0.3929 - learning_rate: 1.0000e-04
Epoch 3/5
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 165s 108ms/step - accuracy: 0.8809 - loss: 0.3003 - val_accuracy: 0.8513 - val_loss: 0.3944 - learning_rate: 1.0000e-04
Epoch 4/5
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 164s 107ms/step - accuracy: 0.9050 - loss: 0.2447 - val_accuracy: 0.8497 - val_loss: 0.4155 - learning_rate: 1.0000e-04
Epoch 5/5
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 166s 108ms/step - accuracy: 0.9251 - loss: 0.2011 - val_accuracy: 0.8356 - val_loss: 0.4519 - learning_rate: 1.0000e-04
513/513 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.8517 - loss: 0.3951
Test Acc.: 85.32%


In [ ]:

new_texts = [
    "perfect money looking safe comfortable reasonably priced hotel, hotel room clean free parking plus easy park, staff helpful nice deal, good location away crowd wharf close, restaurants close walgreens shopping fridge heat microwave,  ",  # 2 review
    "passable nothing special hotel generally clean really needs updating, bathroom strange not typically, shower consists curtain drain floor corner room, terrace beautiful combined bottle wine great way spend afternoon, lack included breakfast left feeling room overpriced compared places stayed italy", # 0
    "worth money rest, hotel not, going big easy relax pampered need stay, going party city, went halloween celebration insane, hotel block bourbon st. friends stay bourbon twice not away noise music clock, nice quiet nap mid day nice prepare nightlife, rooms need updating hotel fairly clean staff nice helpful times, rooms ready early issue toilet kept running fixed 10 minutes, room service tgi friday hold home eat gumbo jumbalya, pool small clean seeing no kids, stay liver recovers" # 2
]

for review in new_texts:
    new_encoding = tokenizer(review, truncation=True, padding=True, return_tensors='tf')

    new_prediction = dnn.predict(new_encoding['input_ids'])

    predicted_label = np.argmax(new_prediction, axis=1)

    print(f"Review: {review}\nPredicted Label: {predicted_label}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Review: perfect money looking safe comfortable reasonably priced hotel, hotel room clean free parking plus easy park, staff helpful nice deal, good location away crowd wharf close, restaurants close walgreens shopping fridge heat microwave,  
Predicted Label: [2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Review: passable nothing special hotel generally clean really needs updating, bathroom strange not typically, shower consists curtain drain floor corner room, terrace beautiful combined bottle wine great way spend afternoon, lack included breakfast left feeling room overpriced compared places stayed italy
Predicted Label: [0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Review: worth money rest, hotel not, going big easy relax pampered need stay, going party city, went halloween celebration insane, hotel block bourbon st. friends stay bourbon twice not away noise music clock, nice quiet nap mid day nice prepare nightlife, rooms need updating hotel fairly cl

2) Выполните предобработку набора данных для подачи на вход нейронной сети в задаче машинного перевода.

In [ ]:
import random
import tensorflow as tf
import string
import re
import pandas as pd
from tensorflow.keras import layers

# Загрузка и подготовка данных из CSV
csv_file = "data_translate.csv"  
df = pd.read_csv(csv_file, sep="\t")  

# Формируем пары (англ, рус) с добавлением токенов начала и конца для русского
pairs = []
for _, row in df.iterrows():
    english = row["en"].strip()
    russian = "[start] " + row["ru"].strip() + " [end]"
    if english and russian:  # Проверяем, что строки не пустые
        pairs.append((english, russian))

# Перемешивание данных
random.shuffle(pairs)

# Разделение на обучающую, валидационную и тестовую выборки
num_val_samples = int(0.15 * len(pairs))
num_train_samples = len(pairs) - 2 * num_val_samples
train_pairs = pairs[:num_train_samples]
val_pairs = pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = pairs[num_train_samples + num_val_samples:]

# Векторизация текста
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "").replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

# Обучение векторизаторов на тренировочных данных
train_english_texts = [pair[0] for pair in train_pairs]
train_russian_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_russian_texts)

# Создание датасета
batch_size = 64

def format_dataset(eng, rus):
    eng = source_vectorization(eng)
    rus = target_vectorization(rus)
    return ({
        "english": eng,
        "russian": rus[:, :-1],
    }, rus[:, 1:])

def make_dataset(pairs):
    # Проверяем, что пары не пустые
    if not pairs:
        raise ValueError("Pairs are empty.")
    
    eng_texts, rus_texts = zip(*pairs)
    
    # Убедимся, что данные в правильном формате
    dataset = tf.data.Dataset.from_tensor_slices((list(eng_texts), list(rus_texts)))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(tf.data.AUTOTUNE).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# Сохранение словарей векторизаторов для обучения
source_vocab = source_vectorization.get_vocabulary()
target_vocab = target_vectorization.get_vocabulary()

# Сохранение в текстовые файлы
with open("source_vocab.txt", "w") as f:
    for word in source_vocab:
        f.write(f"{word}\n")

with open("target_vocab.txt", "w") as f:
    for word in target_vocab:
        f.write(f"{word}\n")


In [ ]:
import random
import tensorflow as tf
import string
import re
import pandas as pd
from tensorflow.keras import layers

# Загрузка и подготовка данных из CSV
csv_file = "data_translate.csv"  
df = pd.read_csv(csv_file, sep="\t")  

# Формируем пары (англ, рус) с добавлением токенов начала и конца для русского
pairs = []
for _, row in df.iterrows():
    english = row["en"].strip()
    russian = "[start] " + row["ru"].strip() + " [end]"
    if english and russian:  # Проверяем, что строки не пустые
        pairs.append((english, russian))

# Перемешивание данных
random.shuffle(pairs)

# Разделение на обучающую, валидационную и тестовую выборки
num_val_samples = int(0.15 * len(pairs))
num_train_samples = len(pairs) - 2 * num_val_samples
train_pairs = pairs[:num_train_samples]
val_pairs = pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = pairs[num_train_samples + num_val_samples:]

# Векторизация текста
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "").replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

# Обучение векторизаторов на тренировочных данных
train_english_texts = [pair[0] for pair in train_pairs]
train_russian_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_russian_texts)

# Создание датасета
batch_size = 64

def format_dataset(eng, rus):
    eng = source_vectorization(eng)
    rus = target_vectorization(rus)
    return ({
        "english": eng,
        "russian": rus[:, :-1],
    }, rus[:, 1:])

def make_dataset(pairs):
    # Проверяем, что пары не пустые
    if not pairs:
        raise ValueError("Pairs are empty.")
    
    eng_texts, rus_texts = zip(*pairs)
    
    # Убедимся, что данные в правильном формате
    dataset = tf.data.Dataset.from_tensor_slices((list(eng_texts), list(rus_texts)))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(tf.data.AUTOTUNE).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# Сохранение словарей векторизаторов для обучения
source_vocab = source_vectorization.get_vocabulary()
target_vocab = target_vectorization.get_vocabulary()

# Сохранение в текстовые файлы
with open("source_vocab.txt", "w") as f:
    for word in source_vocab:
        f.write(f"{word}\n")

with open("target_vocab.txt", "w") as f:
    for word in target_vocab:
        f.write(f"{word}\n")

Transformer

In [ ]:
from tensorflow import keras
import numpy as np

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(query=inputs, value=inputs, key=inputs, attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

class EmbeddedLayer(keras.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.positional_embedding = PositionalEmbedding(sequence_length, input_dim, output_dim)

    def call(self, inputs):
        return self.positional_embedding(inputs)

# Параметры модели
embed_dim = 256
dense_dim = 2048
num_heads = 8
sequence_length = 20  # Пример длины последовательности
vocab_size = 15000  # Размер словаря, соответствующий векторизации

# Входные данные
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
embedded = EmbeddedLayer(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(embedded)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="russian")
embedded_decoder = EmbeddedLayer(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(embedded_decoder, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Создание и компиляция модели
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Обучение модели
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

# Сохранение модели в формате Keras
transformer.save("transformer_model.keras")
print("Модель сохранена в формате Keras.")

# Проверка на тестовых данных
test_ds = make_dataset(test_pairs)

# Оценка модели на тестовых данных
test_loss, test_accuracy = transformer.evaluate(test_ds)
print(f"Тестовая потеря: {test_loss:.4f}, Тестовая точность: {test_accuracy:.4f}")

Epoch 1/30


/home/aa-sablin/env_lab/lib/python3.12/site-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'transformer_encoder_10' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


165/165 ━━━━━━━━━━━━━━━━━━━━ 38s 141ms/step - accuracy: 0.2522 - loss: 5.2702 - val_accuracy: 0.1167 - val_loss: 3.8271
Epoch 2/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.1238 - loss: 3.7388 - val_accuracy: 0.1233 - val_loss: 3.4679
Epoch 3/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.1259 - loss: 3.3839 - val_accuracy: 0.1323 - val_loss: 3.2106
Epoch 4/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.1367 - loss: 2.9837 - val_accuracy: 0.1482 - val_loss: 2.7309
Epoch 5/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.1519 - loss: 2.4952 - val_accuracy: 0.1545 - val_loss: 2.5144
Epoch 6/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.1584 - loss: 2.1911 - val_accuracy: 0.1571 - val_loss: 2.3943
Epoch 7/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step - accuracy: 0.1642 - loss: 1.9488 - val_accuracy: 0.1587 - val_loss: 2.2854
Epoch 8/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step - accuracy: 0.1686 - loss: 1.7573 - val_accuracy: 0.1

In [20]:
import numpy as np
import random

spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))

max_decoded_sentence_length = 20  # максимальная длина для вывода

def decode_sequence(input_sentence):
    # Векторизуем входное предложение (английский текст)
    tokenized_input_sentence = source_vectorization([input_sentence])
    # Начинаем выходную последовательность со стартового токена (пример: "[start]")
    decoded_sentence = "[start]"
    
    for i in range(max_decoded_sentence_length):
        # Токенизируем уже сгенерированное предложение (декодер принимает все токены, кроме последнего)
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        
        # Модель предсказывает следующий токен
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        
        # Берём вероятности для следующего токена на позиции i
        predicted_id = np.argmax(predictions[0, i, :])
        
        # Получаем слово по индексу
        predicted_word = spa_index_lookup.get(predicted_id, "[unk]")
        
        # Добавляем слово к выходной последовательности
        decoded_sentence += " " + predicted_word
        
        # Останавливаемся, если встретили токен "[end]"
        if predicted_word == "[end]":
            break
    
    return decoded_sentence

# Демонстрация вывода 20 различных примеров из тестового набора
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-" * 30)
    print("Input:", input_sentence)
    print("Output:", decode_sequence(input_sentence))


------------------------------
Input: I need it now.
Output: [start] Мне нужно сейчас [end]
------------------------------
Input: Check this out.
Output: [start] Проверьте это [end]
------------------------------
Input: Get in line.
Output: [start] Будьте на строчку [end]
------------------------------
Input: Sit down.
Output: [start] Сядь [end]
------------------------------
Input: They hired me.
Output: [start] Они взяли на работу [end]
------------------------------
Input: Be realistic.
Output: [start] Будь реалистом [end]
------------------------------
Input: Get a job.
Output: [start] Найди работу [end]
------------------------------
Input: It's outdated.
Output: [start] Это устарело [end]
------------------------------
Input: Put it down.
Output: [start] Поставьте её [end]
------------------------------
Input: I admire you.
Output: [start] Я восхищаюсь Томом [end]
------------------------------
Input: Let's eat out.
Output: [start] Давай пойдём куданибудь поедим [end]
-----------

RNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

embedding_dim = 256
units = 512

# Создаем модель 
input_english = layers.Input(shape=(sequence_length,), name="english")
input_russian = layers.Input(shape=(sequence_length,), name="russian")

# Встраивание английских слов
x = layers.Embedding(input_dim=len(source_vocab), output_dim=embedding_dim, mask_zero=True)(input_english)

# Кодировщик
encoder_output = layers.LSTM(units)(x)

# Встраивание русских слов (вход декодера)
decoder_embedding = layers.Embedding(input_dim=len(target_vocab), output_dim=embedding_dim, mask_zero=True)(input_russian)

# Декодер с инициализацией состояния из кодировщика
decoder_lstm = layers.LSTM(units, return_sequences=True)
decoder_output = decoder_lstm(decoder_embedding, initial_state=[encoder_output, encoder_output])

# Выходной слой
output = layers.Dense(len(target_vocab), activation='softmax')(decoder_output)

model = models.Model(inputs=[input_english, input_russian], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели 
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    verbose=1
)

# Сохранение модели в формате Keras
transformer.save("RNN_model.keras")
print("Модель сохранена в формате Keras.")



Epoch 1/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.3817 - loss: 5.4594 - val_accuracy: 0.1209 - val_loss: 3.5592
Epoch 2/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.1218 - loss: 3.3540 - val_accuracy: 0.1277 - val_loss: 3.2896
Epoch 3/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.1293 - loss: 2.9866 - val_accuracy: 0.1343 - val_loss: 3.1037
Epoch 4/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.1350 - loss: 2.6644 - val_accuracy: 0.1367 - val_loss: 3.0080
Epoch 5/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.1392 - loss: 2.3787 - val_accuracy: 0.1396 - val_loss: 2.9084
Epoch 6/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.1435 - loss: 2.1107 - val_accuracy: 0.1424 - val_loss: 2.8352
Epoch 7/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.1480 - loss: 1.8601 - val_accuracy: 0.1441 - val_loss: 2.7825
Epoch 8/30
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.1531 - loss: 1.6344 - val_accu

In [27]:
import numpy as np

# Создаем словарь для обратного поиска слова по индексу для русского
target_vocab = target_vectorization.get_vocabulary()
target_index_to_word = dict(enumerate(target_vocab))

max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    # Преобразуем вход в вектор токенов
    tokenized_input = source_vectorization([input_sentence])

    # Начинаем со стартового токена [start]
    decoded_sentence = "[start]"
    
    for i in range(max_decoded_sentence_length):
        # Преобразуем текущую часть предложения-декодера
        tokenized_target = target_vectorization([decoded_sentence])
        
        # Предсказываем вероятности следующего слова
        predictions = model.predict([tokenized_input, tokenized_target])
        
        # Выбираем токен с наибольшей вероятностью для текущего шага i
        next_token_idx = np.argmax(predictions[0, i, :])
        next_word = target_index_to_word[next_token_idx]
        
        # Добавляем токен к ответу
        decoded_sentence += " " + next_word
        
        # Если достигли токена конца предложения, прерываем
        if next_word == "[end]":
            break

    # Возвращаем срез без стартового и конечного токена
    decoded_words = decoded_sentence.split()[1:-1]
    return " ".join(decoded_words)

# Пример применения:
test_english_sentences = [pair[0] for pair in test_pairs]

import random

for _ in range(20):
    inp = random.choice(test_english_sentences)
    print("Input: ", inp)
    print("Output:", decode_sequence(inp))
    print("-" * 40)


Input:  I'm wasted.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Output: Я измождена
----------------------------------------
Input:  Is Tom coming?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Output: Том идёт
----------------------------------------
Input:  Wait a moment.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Output: Погодите минутку
----------------------------------------
Input:  Nobody slept.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Output: Никто не шелохнулся
----------------------------------------
Input:  I warned you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

Как видно из сравнения, модель Transformer показывает лучшие результаты: она лучше справляется с контекстуальными особенностями перевода и даёт более точный перевод в сложных случаях.
Время обучения отличается достаточно сильно — примерно на 45 процентов в пользу RNN. Так, среднее время обучения RNN составляет 3,11 минуты, а время обучения Transformer — 4,52 минуты.
В заключении хотелось бы сказать, что трансформер работает намного лучше, чем RNN, в задаче машинного перевода. Это связано с весомыми преимуществами архитектуры трансформера, которые 
позволяют ему эффективно обрабатывать сложные зависимости и длинные контексты.



In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1
